This notebook demonstrate support of sequential and functional model of `tf.keras` on ktrain.

In [1]:
import ktrain
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPool1D, BatchNormalization, Input, Concatenate

In [2]:
class_names = ['alt.atheism', 'comp.sys.ibm.pc.hardware', 'sci.space', 'talk.politics.guns']
train = fetch_20newsgroups(subset='train', remove=['headers', 'footers', 'quotes'], categories=class_names, shuffle=False)
test = fetch_20newsgroups(subset='test', remove=['headers', 'footers', 'quotes'], categories=class_names, shuffle=False)

In [3]:
train_ds, test_ds, preproc = ktrain.text.texts_from_array(
    x_train=train.data, y_train=train.target,
    x_test=test.data, y_test=test.target,
    class_names=class_names, maxlen=500,
    max_features=20000, preprocess_mode='standard'
)

language: en
Word Counts: 30716
Nrows: 2209
2209 train sequences
train sequence lengths:
	mean : 187
	95percentile : 600
	99percentile : 1892
x_train shape: (2209,500)
y_train shape: (2209, 4)
Is Multi-Label? False
1469 test sequences
test sequence lengths:
	mean : 152
	95percentile : 492
	99percentile : 1144
x_test shape: (1469,500)
y_test shape: (1469, 4)
task: text classification


In [4]:
model = Sequential([
    Embedding(20000, 100, input_length=500),
    GlobalAveragePooling1D(),
    BatchNormalization(),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          2000000   
                                                                 
 global_average_pooling1d (G  (None, 100)              0         
 lobalAveragePooling1D)                                          
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 2,000,804
Trainable params: 2,000,604
Non-trainable params: 200
_________________________________________________________________


In [5]:
learner = ktrain.get_learner(model, train_data=train_ds, val_data=test_ds, batch_size=64)
learner.autofit(0.01, 5, verbose=2)



begin training using triangular learning rate policy with max lr of 0.01...
Epoch 1/5
35/35 - 2s - loss: 0.9858 - accuracy: 0.6609 - val_loss: 1.3399 - val_accuracy: 0.3029 - 2s/epoch - 47ms/step
Epoch 2/5
35/35 - 0s - loss: 0.4106 - accuracy: 0.8882 - val_loss: 1.3233 - val_accuracy: 0.3036 - 237ms/epoch - 7ms/step
Epoch 3/5
35/35 - 0s - loss: 0.2995 - accuracy: 0.9036 - val_loss: 1.2746 - val_accuracy: 0.3288 - 239ms/epoch - 7ms/step
Epoch 4/5
35/35 - 0s - loss: 0.1971 - accuracy: 0.9339 - val_loss: 1.1722 - val_accuracy: 0.4118 - 239ms/epoch - 7ms/step
Epoch 5/5
35/35 - 0s - loss: 0.1340 - accuracy: 0.9574 - val_loss: 1.0905 - val_accuracy: 0.5759 - 281ms/epoch - 8ms/step


In [6]:
inp = Input(500)
emb = Embedding(20000, 100, input_length=500)(inp)
gap = GlobalAveragePooling1D()(emb)
gmp = GlobalMaxPool1D()(emb)
con = Concatenate()([gap, gmp])
bn = BatchNormalization()(con)
out = Dense(4, activation='softmax')(bn)

model = Model(inputs=inp, outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 500, 100)     2000000     ['input_1[0][0]']                
                                                                                                  
 global_average_pooling1d_1 (Gl  (None, 100)         0           ['embedding_1[0][0]']            
 obalAveragePooling1D)                                                                            
                                                                                                  
 global_max_pooling1d (GlobalMa  (None, 100)         0           ['embedding_1[0][0]']        

In [7]:
learner = ktrain.get_learner(model, train_data=train_ds, val_data=test_ds, batch_size=64)
learner.autofit(0.002, 5, verbose=2)



begin training using triangular learning rate policy with max lr of 0.002...
Epoch 1/5
35/35 - 1s - loss: 1.1127 - accuracy: 0.6202 - val_loss: 1.3497 - val_accuracy: 0.4935 - 910ms/epoch - 26ms/step
Epoch 2/5
35/35 - 0s - loss: 0.4680 - accuracy: 0.9276 - val_loss: 1.2981 - val_accuracy: 0.3799 - 262ms/epoch - 7ms/step
Epoch 3/5
35/35 - 0s - loss: 0.1956 - accuracy: 0.9719 - val_loss: 1.2467 - val_accuracy: 0.5194 - 302ms/epoch - 9ms/step
Epoch 4/5
35/35 - 0s - loss: 0.1042 - accuracy: 0.9765 - val_loss: 1.1983 - val_accuracy: 0.6120 - 272ms/epoch - 8ms/step
Epoch 5/5
35/35 - 0s - loss: 0.0734 - accuracy: 0.9814 - val_loss: 1.1463 - val_accuracy: 0.6767 - 287ms/epoch - 8ms/step
